In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

23/03/29 11:25:18 WARN Utils: Your hostname, DavidNg-8242-NB resolves to a loopback address: 127.0.1.1; using 172.24.225.125 instead (on interface eth0)
23/03/29 11:25:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/29 11:25:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
import pandas as pd

In [66]:
df = spark.read \
    .option("header", "true") \
    .option("multiline", True) \
    .option("escape", "\"") \
    .csv('data/raw/china/hk/hong-kong/2022/12/listings_20221223.csv.gz')

In [67]:
df.limit(5).toPandas()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,17891,https://www.airbnb.com/rooms/17891,20221223064914,2022-12-23,previous scrape,Large Light Filled Loft,"Gorgeous and spacious loft, in the best locati...",Best neighborhood in Hong Kong! A mix of old a...,https://a0.muscache.com/pictures/119686/6ced58...,69063,...,4.93,4.9,4.66,None,f,1,1,0,0,0.47
1,72571,https://www.airbnb.com/rooms/72571,20221223064914,2022-12-23,city scrape,SoHo off Hollywood Road 3A,"Located steps from Hollywood road, with new ea...",None,https://a0.muscache.com/pictures/2849554/d2262...,304876,...,4.51,4.73,4.13,None,f,10,6,4,0,1.05
2,923017,https://www.airbnb.com/rooms/923017,20221223064914,2022-12-23,city scrape,Comfort Home in Prince Edward Mtr 4 beds,A newly renovated apartment located in the ci...,Disneyland Discount ticket<br />Ocean Park Di...,https://a0.muscache.com/pictures/f3913158-fa84...,4971086,...,4.85,4.86,4.75,None,t,2,2,0,0,3.03
3,944283,https://www.airbnb.com/rooms/944283,20221223064914,2022-12-23,city scrape,2 Bed Rooms heart of Central SOHO( sleeps 3),"Soho, best Location in Central. Hong Kong.<br ...",Amazing location on Wellington street in Centr...,https://a0.muscache.com/pictures/9ade4292-c83a...,5100812,...,4.72,4.85,4.57,None,f,2,1,1,0,0.61
4,956857,https://www.airbnb.com/rooms/956857,20221223064914,2022-12-23,city scrape,large Room- Heart of Soho/Central,Awesome Location in Central.<br />Clean large...,None,https://a0.muscache.com/pictures/3ab4ff36-81de...,5100812,...,5.0,5.0,5.0,None,f,2,1,1,0,0.04


In [18]:
df.schema

StructType([StructField('id', StringType(), True), StructField('listing_url', StringType(), True), StructField('scrape_id', StringType(), True), StructField('last_scraped', StringType(), True), StructField('source', StringType(), True), StructField('name', StringType(), True), StructField('description', StringType(), True), StructField('neighborhood_overview', StringType(), True), StructField('picture_url', StringType(), True), StructField('host_id', StringType(), True), StructField('host_url', StringType(), True), StructField('host_name', StringType(), True), StructField('host_since', StringType(), True), StructField('host_location', StringType(), True), StructField('host_about', StringType(), True), StructField('host_response_time', StringType(), True), StructField('host_response_rate', StringType(), True), StructField('host_acceptance_rate', StringType(), True), StructField('host_is_superhost', StringType(), True), StructField('host_thumbnail_url', StringType(), True), StructField('

In [11]:
from pyspark.sql import types

In [12]:
hk_schema = types.StructType([
    types.StructField('id', types.StringType(), True), 
    types.StructField('listing_url', types.StringType(), True), 
    types.StructField('scrape_id', types.StringType(), True), 
    types.StructField('last_scraped', types.StringType(), True), 
    types.StructField('source', types.StringType(), True), 
    types.StructField('name', types.StringType(), True), 
    types.StructField('description', types.StringType(), True), 
    types.StructField('neighborhood_overview', types.StringType(), True), 
    types.StructField('picture_url', types.StringType(), True), 
    types.StructField('host_id', types.StringType(), True), 
    types.StructField('host_url', types.StringType(), True), 
    types.StructField('host_name', types.StringType(), True), 
    types.StructField('host_since', types.StringType(), True), 
    types.StructField('host_location', types.StringType(), True), 
    types.StructField('host_about', types.StringType(), True), 
    types.StructField('host_response_time', types.StringType(), True), 
    types.StructField('host_response_rate', types.StringType(), True), 
    types.StructField('host_acceptance_rate', types.StringType(), True), 
    types.StructField('host_is_superhost', types.StringType(), True), 
    types.StructField('host_thumbnail_url', types.StringType(), True), 
    types.StructField('host_picture_url', types.StringType(), True), 
    types.StructField('host_neighbourhood', types.StringType(), True), 
    types.StructField('host_listings_count', types.StringType(), True), 
    types.StructField('host_total_listings_count', types.StringType(), True), 
    types.StructField('host_verifications', types.StringType(), True), 
    types.StructField('host_has_profile_pic', types.StringType(), True), 
    types.StructField('host_identity_verified', types.StringType(), True), 
    types.StructField('neighbourhood', types.StringType(), True), 
    types.StructField('neighbourhood_cleansed', types.StringType(), True), 
    types.StructField('neighbourhood_group_cleansed', types.StringType(), True), 
    types.StructField('latitude', types.StringType(), True), 
    types.StructField('longitude', types.StringType(), True), 
    types.StructField('property_type', types.StringType(), True), 
    types.StructField('room_type', types.StringType(), True), 
    types.StructField('accommodates', types.StringType(), True), 
    types.StructField('bathrooms', types.StringType(), True), 
    types.StructField('bathrooms_text', types.StringType(), True), 
    types.StructField('bedrooms', types.StringType(), True), 
    types.StructField('beds', types.StringType(), True), 
    types.StructField('amenities', types.StringType(), True), 
    types.StructField('price', types.StringType(), True), 
    types.StructField('minimum_nights', types.IntegerType(), True), 
    types.StructField('maximum_nights', types.IntegerType(), True), 
    types.StructField('minimum_minimum_nights', types.FloatType(), True), 
    types.StructField('maximum_minimum_nights', types.FloatType(), True), 
    types.StructField('minimum_maximum_nights', types.FloatType(), True), 
    types.StructField('maximum_maximum_nights', types.FloatType(), True), 
    types.StructField('minimum_nights_avg_ntm', types.FloatType(), True), 
    types.StructField('maximum_nights_avg_ntm', types.FloatType(), True), 
    types.StructField('calendar_updated', types.StringType(), True), 
    types.StructField('has_availability', types.StringType(), True), 
    types.StructField('availability_30', types.IntegerType(), True), 
    types.StructField('availability_60', types.IntegerType(), True), 
    types.StructField('availability_90', types.IntegerType(), True), 
    types.StructField('availability_365', types.IntegerType(), True), 
    types.StructField('calendar_last_scraped', types.StringType(), True), 
    types.StructField('number_of_reviews', types.IntegerType(), True), 
    types.StructField('number_of_reviews_ltm', types.IntegerType(), True), 
    types.StructField('number_of_reviews_l30d', types.IntegerType(), True), 
    types.StructField('first_review', types.StringType(), True), 
    types.StructField('last_review', types.StringType(), True), 
    types.StructField('review_scores_rating', types.FloatType(), True), 
    types.StructField('review_scores_accuracy', types.FloatType(), True), 
    types.StructField('review_scores_cleanliness', types.FloatType(), True), 
    types.StructField('review_scores_checkin', types.FloatType(), True), 
    types.StructField('review_scores_communication', types.FloatType(), True), 
    types.StructField('review_scores_location', types.FloatType(), True), 
    types.StructField('review_scores_value', types.FloatType(), True), 
    types.StructField('license', types.StringType(), True), 
    types.StructField('instant_bookable', types.StringType(), True), 
    types.StructField('calculated_host_listings_count', types.FloatType(), True), 
    types.StructField('calculated_host_listings_count_entire_homes', types.FloatType(), True), 
    types.StructField('calculated_host_listings_count_private_rooms', types.FloatType(), True), 
    types.StructField('calculated_host_listings_count_shared_rooms', types.FloatType(), True), 
    types.StructField('reviews_per_month', types.FloatType(), True)])

In [27]:
year = 2022

for month in range(3, 13, 3):
    print(f'processing data for {year}/{month}')

    input_path = f'data/raw/china/hk/hong-kong/{year}/{month:02d}/'
    output_path = f'data/pq/china/hk/hong-kong/{year}/{month:02d}/'

    df_hk = spark.read \
        .option("header", "true") \
        .schema(hk_schema) \
        .csv(input_path)

    df_hk \
        .repartition(1) \
        .write.parquet(output_path, mode='overwrite')

processing data for 2022/3
23/03/30 17:50:20 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 74, schema size: 75
CSV file: file:///home/derekwong/dataengineering_zoomcamp2023/week7/data/raw/china/hk/hong-kong/2022/03/listings_20220321.csv.gz


processing data for 2022/6
23/03/30 17:50:24 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 74, schema size: 75
CSV file: file:///home/derekwong/dataengineering_zoomcamp2023/week7/data/raw/china/hk/hong-kong/2022/06/listings_20220614.csv.gz


processing data for 2022/9


processing data for 2022/12


In [41]:
df_all = spark.read.parquet('data/pq/china/hk/hong-kong/2022/12/')

In [68]:
df_all.head()

Row(id='17891', listing_url='https://www.airbnb.com/rooms/17891', scrape_id='20221223064914', last_scraped='2022-12-23', source='previous scrape', name='Large Light Filled Loft', description="Gorgeous and spacious loft, in the best location. Also available for Commercial Shoots and Longer term rentals.<br /><br /><b>The space</b><br />The apartment has been featured on the cover of numerous Design Magazines, and is listed as one of Airbnb's Top 40 Rentals Worldwide. <br /><br />Longer Term / Monthly rates available. <br />Also available for Commercial Shoots, inquire for day rates. <br /><br />Large & light-filled, designer open floor plan (800 sqf), fully furnished, with ultra-modern, open kitchen , available for short & long term. <br /><br />Includes:<br />- full kitchen (stove top, oven)<br />- fully equipped workstation with wi-fi & printer<br />- balcony & large windows<br />- queen size bed<br />- walk-in wardrobe<br />- 10 ft dining table perfect for entertaining<br />- polishe